In [7]:
import polars as pl
import datetime as dt
import sf_quant.data as sfd

In [8]:
start = dt.date(2000, 1, 1)
end = dt.date(2024, 12, 31)
price_filter = 5
IC = 0.05

In [9]:
data = sfd.load_assets(
    start=start,
    end=end,
    columns=[
        "date",
        "barrid",
        "cusip",
        "price",
        "return",
        "specific_return",
        "specific_risk",
        "predicted_beta",
        "historical_beta",
        "market_cap",
    ],
    in_universe=True,
).with_columns(pl.col("return", "specific_risk").truediv(100))

In [10]:
# 1. Load Compustat (Monthly/Quarterly Data)
compustat = pl.read_csv("data/data.csv").with_columns(
    pl.col("datadate").str.strptime(pl.Date, "%Y-%m-%d")
).select([
    pl.col("GVKEY").alias("gvkey"),  # Rename immediately for consistency
    pl.col("datadate").alias("date"),
    "tic",
    "gind"
]).sort("date")  # asof joins require sorted keys!

# 2. Load Link Table
links = pl.read_csv("data/links.csv").with_columns([
    pl.col("LINKDT").str.strptime(pl.Date, "%Y-%m-%d"),
    pl.col("LINKENDDT").replace("E", "2099-12-31").str.strptime(pl.Date, "%Y-%m-%d"),
])

# 3. Link GVKEY to Daily Data (Range Join)
# Note: This step is usually fine as an inner join if you just want to map IDs.
# We join on CUSIP first, then filter by date range.
data = data.join(
    links,
    left_on="cusip", 
    right_on="cusip",
    how="inner"
).filter(
    pl.col("date").is_between(pl.col("LINKDT"), pl.col("LINKENDDT"))
).sort("date") # Sort daily data for asof join

# asof join
# merges the 'nearest' previous Compustat date to each daily price date.
data = data.join_asof(
    compustat,
    on="date",           # The time key
    by="gvkey",          # The entity key (must match in both DFs)
    strategy="backward"  # Look for the most recent past date
)

# de-duplicate: may resolve a shape mismatch error?
data = data.unique(subset=["date", "barrid"], keep="first")

/tmp/ipykernel_3125615/3156332614.py:31: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  data = data.join_asof(


In [12]:
data['date'].unique().sort()

date
date
2000-01-03
2000-01-04
2000-01-05
2000-01-06
2000-01-07
…
2024-12-24
2024-12-26
2024-12-27


In [13]:
# ---------------------------------------------------------
# 1. AGGREGATE TO INDUSTRY LEVEL
# ---------------------------------------------------------
# We create a temporary DF to calculate industry weights and returns safely
industry_df = (
    data
    .group_by(["date", "gind"])
    .agg([
        pl.col("market_cap").sum().alias("industry_mkt_cap"),
        
        # Calculate weighted average specific return/risk
        # Note: We need to do the weighting inside the aggregation
        (pl.col("market_cap") * pl.col("specific_return")).sum().alias("sum_w_ret"),
        (pl.col("market_cap") * pl.col("specific_risk")).sum().alias("sum_w_risk")
    ])
    .with_columns([
        (pl.col("sum_w_ret") / pl.col("industry_mkt_cap")).alias("ind_spec_ret"),
        (pl.col("sum_w_risk") / pl.col("industry_mkt_cap")).alias("ind_spec_risk")
    ])
    .sort(["gind", "date"]) # Vital for rolling functions
)

In [14]:
industry_df

date,gind,industry_mkt_cap,sum_w_ret,sum_w_risk,ind_spec_ret,ind_spec_risk
date,i64,f64,f64,f64,f64,f64
2000-01-03,null,9.5924e12,1.2580e12,3.1187e12,0.13114,0.325118
2000-01-04,null,9.2273e12,1.5827e12,2.9874e12,0.171525,0.323754
2000-01-05,null,9.2354e12,3.9500e12,2.9878e12,0.427702,0.323517
2000-01-06,null,9.1880e12,6.9876e11,2.9573e12,0.076052,0.321871
2000-01-07,null,9.4758e12,-6.6549e11,3.0618e12,-0.070231,0.323113
…,…,…,…,…,…,…
2024-12-24,602010,1.2006e11,-4.6375e10,3.1228e10,-0.38628,0.260108
2024-12-26,602010,1.2012e11,-8.9775e9,3.1216e10,-0.074735,0.259867
2024-12-27,602010,1.1872e11,3.8372e10,3.0782e10,0.323204,0.25928


In [ ]:
# ---------------------------------------------------------
# 2. CALCULATE MOMENTUM ON INDUSTRY DF
# ---------------------------------------------------------
industry_df = industry_df.with_columns([
    pl.col("ind_spec_ret")
    .rolling_sum(window_size=231)
    .shift(21) # 1-month lag
    .over("gind")
    .alias("raw_ind_mom")
])

industry_df = industry_df.with_columns([
    (pl.col("raw_ind_mom") / pl.col("ind_spec_risk")).alias("vol_adj_ind_mom")
])

In [16]:
industry_df

date,gind,industry_mkt_cap,sum_w_ret,sum_w_risk,ind_spec_ret,ind_spec_risk,raw_ind_mom,vol_adj_ind_mom
date,i64,f64,f64,f64,f64,f64,f64,f64
2000-01-03,null,9.5924e12,1.2580e12,3.1187e12,0.13114,0.325118,null,null
2000-01-04,null,9.2273e12,1.5827e12,2.9874e12,0.171525,0.323754,null,null
2000-01-05,null,9.2354e12,3.9500e12,2.9878e12,0.427702,0.323517,null,null
2000-01-06,null,9.1880e12,6.9876e11,2.9573e12,0.076052,0.321871,null,null
2000-01-07,null,9.4758e12,-6.6549e11,3.0618e12,-0.070231,0.323113,null,null
…,…,…,…,…,…,…,…,…
2024-12-24,602010,1.2006e11,-4.6375e10,3.1228e10,-0.38628,0.260108,4.96726,19.096887
2024-12-26,602010,1.2012e11,-8.9775e9,3.1216e10,-0.074735,0.259867,6.239025,24.008502
2024-12-27,602010,1.1872e11,3.8372e10,3.0782e10,0.323204,0.25928,5.543845,21.381672


In [17]:
# ---------------------------------------------------------
# 3. MERGE BACK TO STOCK LEVEL
# ---------------------------------------------------------
data = data.join(
    industry_df.select(["date", "gind", "raw_ind_mom", "vol_adj_ind_mom", "ind_spec_risk"]),
    on=["date", "gind"],
    how="left"
)

In [20]:
data['date'].unique().sort()

date
date
2000-01-03
2000-01-04
2000-01-05
2000-01-06
2000-01-07
…
2024-12-24
2024-12-26
2024-12-27


In [21]:
# ---------------------------------------------------------
# 4. SCORING & ALPHA (Stock Level)
# ---------------------------------------------------------
# Now we are back at the stock level, we can score cross-sectionally
data = data.with_columns([
    # Z-Score Vol-Adj Momentum
    (pl.col("vol_adj_ind_mom") - pl.col("vol_adj_ind_mom").mean())
    .truediv(pl.col("vol_adj_ind_mom").std())
    .over("date")
    .clip(-3.0, 3.0)
    .alias("score_vol_adj"),

    # Z-Score Raw Momentum
    (pl.col("raw_ind_mom") - pl.col("raw_ind_mom").mean())
    .truediv(pl.col("raw_ind_mom").std())
    .over("date")
    .clip(-3.0, 3.0)
    .alias("score_raw")
])

In [22]:
# Calculate Alphas
# Note: Using 'ind_spec_risk' here means we scale by Industry Risk.
# If you wanted to scale by Stock Risk, use 'specific_risk'.
data = data.with_columns([
    (pl.col("ind_spec_risk") * IC * pl.col("score_vol_adj")).alias("alpha_vol_adj_mom"),
    (pl.col("ind_spec_risk") * IC * pl.col("score_raw")).alias("alpha_raw_mom")
])

In [26]:
data

date,barrid,cusip,price,return,specific_return,specific_risk,predicted_beta,historical_beta,market_cap,gvkey,tic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,tic_right,gind,raw_ind_mom,vol_adj_ind_mom,ind_spec_risk,score_vol_adj,score_raw,alpha_vol_adj_mom,alpha_raw_mom
date,str,str,f64,f64,f64,f64,f64,f64,f64,i64,str,str,str,str,i64,i64,date,date,str,i64,f64,f64,f64,f64,f64,f64,f64
2013-07-31,"""USA06Z1""","""602496101""",6.26,-0.001595,-0.788,0.550569,0.34349,0.328385,6.006157e8,178157,"""MDXG""","""P""","""01""","""LC""",13878,54378,2013-04-25,2018-11-07,null,null,null,null,null,null,null,null,null
2013-08-01,"""USA06Z1""","""602496101""",6.32,0.009585,0.365,0.55028,0.353329,0.334989,6.0865392e8,178157,"""MDXG""","""P""","""01""","""LC""",13878,54378,2013-04-25,2018-11-07,null,null,null,null,null,null,null,null,null
2013-08-02,"""USA06Z1""","""602496101""",6.31,-0.001582,-0.731,0.548074,0.363624,0.330713,6.0769086e8,178157,"""MDXG""","""P""","""01""","""LC""",13878,54378,2013-04-25,2018-11-07,null,null,null,null,null,null,null,null,null
2013-08-05,"""USA06Z1""","""602496101""",6.45,0.022187,1.936,0.547667,0.356596,0.324494,6.211737e8,178157,"""MDXG""","""P""","""01""","""LC""",13878,54378,2013-04-25,2018-11-07,null,null,null,null,null,null,null,null,null
2013-08-06,"""USA06Z1""","""602496101""",6.29,-0.024806,-0.528,0.546922,0.399196,0.366323,6.0576474e8,178157,"""MDXG""","""P""","""01""","""LC""",13878,54378,2013-04-25,2018-11-07,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-24,"""USBQOR1""","""300426103""",70.58,0.025872,2.218,0.268004,1.287294,1.370328,3.5976e9,50838,"""ECG""","""P""","""01""","""LC""",25661,60018,2024-11-01,2099-12-31,"""ECG""",201030,3.884854,17.128539,0.226806,-0.142745,-0.13828,-0.001619,-0.001568
2024-12-26,"""USBQOR1""","""300426103""",73.61,0.04293,3.633,0.271723,1.288943,1.363927,3.7521e9,50838,"""ECG""","""P""","""01""","""LC""",25661,60018,2024-11-01,2099-12-31,"""ECG""",201030,3.954401,17.419452,0.227011,-0.147299,-0.141286,-0.001672,-0.001604
2024-12-27,"""USBQOR1""","""300426103""",69.85,-0.05108,-3.046,0.274681,1.294801,1.366608,3.5604e9,50838,"""ECG""","""P""","""01""","""LC""",25661,60018,2024-11-01,2099-12-31,"""ECG""",201030,4.208443,18.577704,0.226532,-0.124455,-0.117217,-0.00141,-0.001328


In [24]:
data['date'].unique().sort()

date
date
2000-01-03
2000-01-04
2000-01-05
2000-01-06
2000-01-07
…
2024-12-24
2024-12-26
2024-12-27


In [25]:
# ---------------------------------------------------------
# 5. FILTERING
# ---------------------------------------------------------
# Sort by Barrid/Date to ensure shift(1) works correctly for Price
data = data.sort(["barrid", "date"])

for signal_name in ["alpha_vol_adj_mom", "alpha_raw_mom"]:
    print(f"Processing {signal_name}...")
    
    lean_df = (
        data
        .filter(
            # Using 'is_not_null' avoids errors if previous day is missing
            pl.col("price").shift(1).over("barrid") > price_filter
        )
        .filter(
            pl.col(signal_name).is_not_null(),
            pl.col("predicted_beta").is_not_null()
        )
        .select(["date", "barrid", "predicted_beta", pl.col(signal_name).alias("alpha")])
    )
    
    filename = f"{signal_name}_data.parquet"
    lean_df.write_parquet(filename)
    
    print(f" -> Wrote {lean_df.height} rows to {filename}")
    print(f" -> Columns: {lean_df.columns}")

Processing alpha_vol_adj_mom...
 -> Wrote 4765951 rows to alpha_vol_adj_mom_data.parquet
 -> Columns: ['date', 'barrid', 'predicted_beta', 'alpha']
Processing alpha_raw_mom...
 -> Wrote 4765951 rows to alpha_raw_mom_data.parquet
 -> Columns: ['date', 'barrid', 'predicted_beta', 'alpha']
